# Tier 0: Pricing Foundations

**Learning Objectives:**
- Understand the difference between physical and risk-neutral measures
- Learn the Black-Scholes framework basics
- Get familiar with annuity-pricing library API

**Prerequisites:**
- Basic Python knowledge
- Familiarity with options concepts (calls, puts)
- Understanding of present value / discounting

**Duration:** ~30 minutes

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt

# Library imports
from annuity_pricing.options.pricing.black_scholes import (
    black_scholes_call,
    black_scholes_put,
)

print("Setup complete!")

## 1. Physical vs Risk-Neutral Measures

### The Key Question

When pricing options, we need to answer: **What return should we use for the underlying?**

Two possible answers:

| Measure | Drift | Used For |
|---------|-------|----------|
| **Physical (P)** | Historical average μ | Forecasting, risk management |
| **Risk-Neutral (Q)** | Risk-free rate r | Pricing, hedging |

### Why Risk-Neutral? [T1]

Option prices are determined by **arbitrage**, not expectations:
- A call option can be replicated by a portfolio of stock + bond
- The replication cost determines the fair price
- This price is **independent of investor preferences**

Under the risk-neutral measure Q:
- Expected return = risk-free rate r (minus dividend yield q)
- All assets have the same expected return (r)
- Prices are discounted expectations under Q

In [ ]:
# Parameters
S = 100.0      # Spot price
K = 100.0      # Strike (ATM)
r = 0.05       # Risk-free rate (5%)
q = 0.02       # Dividend yield (2%)
sigma = 0.20   # Volatility (20%)
T = 1.0        # Time to expiry (1 year)

# Historical return (often higher than risk-free for equities)
mu_historical = 0.10  # 10% historical average

print("Market Parameters:")
print(f"  Spot (S) = ${S:.2f}")
print(f"  Strike (K) = ${K:.2f}")
print(f"  Risk-free rate (r) = {r:.1%}")
print(f"  Dividend yield (q) = {q:.1%}")
print(f"  Volatility (σ) = {sigma:.1%}")
print(f"  Time (T) = {T} year")
print()
print(f"  Historical return (μ) = {mu_historical:.1%}")
print(f"  Risk-neutral drift (r-q) = {r-q:.1%}")

## 2. Black-Scholes Pricing

The Black-Scholes formula prices European options under these assumptions [T1]:

1. **Log-normal returns**: Stock prices follow geometric Brownian motion
2. **Constant volatility**: σ doesn't change over time
3. **No arbitrage**: Can buy/sell freely, no transaction costs
4. **Continuous trading**: Can hedge continuously

### Call Price Formula

$$C = S e^{-qT} N(d_1) - K e^{-rT} N(d_2)$$

where:
$$d_1 = \frac{\ln(S/K) + (r - q + \frac{1}{2}\sigma^2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1 - \sigma\sqrt{T}$$

In [ ]:
# Price a call option
call_price = black_scholes_call(S, K, r, q, sigma, T)
put_price = black_scholes_put(S, K, r, q, sigma, T)

print(f"Black-Scholes Prices:")
print(f"  Call = ${call_price:.4f}")
print(f"  Put  = ${put_price:.4f}")

## 3. Put-Call Parity

A fundamental relationship that MUST hold [T1]:

$$C - P = S e^{-qT} - K e^{-rT}$$

This is an arbitrage relationship, not a model assumption.
If your implementation violates this, you have a bug!

In [ ]:
# Verify put-call parity
lhs = call_price - put_price
rhs = S * np.exp(-q * T) - K * np.exp(-r * T)

print("Put-Call Parity Check:")
print(f"  C - P           = ${lhs:.6f}")
print(f"  S*exp(-qT) - K*exp(-rT) = ${rhs:.6f}")
print(f"  Difference      = ${abs(lhs - rhs):.2e}")
print(f"  Status: {'✓ PASS' if abs(lhs - rhs) < 1e-10 else '✗ FAIL'}")

## 4. Option Payoffs

Let's visualize the payoff structure of calls and puts.

In [ ]:
# Range of spot prices at expiry
S_range = np.linspace(50, 150, 100)

# Payoffs at expiry
call_payoff = np.maximum(S_range - K, 0)
put_payoff = np.maximum(K - S_range, 0)

# Current prices
call_prices = [black_scholes_call(s, K, r, q, sigma, T) for s in S_range]
put_prices = [black_scholes_put(s, K, r, q, sigma, T) for s in S_range]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Call option
axes[0].plot(S_range, call_payoff, 'b--', label='Payoff at Expiry')
axes[0].plot(S_range, call_prices, 'b-', label='Current Price')
axes[0].axhline(y=0, color='gray', linestyle='-', alpha=0.3)
axes[0].axvline(x=K, color='red', linestyle=':', alpha=0.5, label=f'Strike K={K}')
axes[0].set_xlabel('Spot Price')
axes[0].set_ylabel('Value ($)')
axes[0].set_title('Call Option')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Put option
axes[1].plot(S_range, put_payoff, 'r--', label='Payoff at Expiry')
axes[1].plot(S_range, put_prices, 'r-', label='Current Price')
axes[1].axhline(y=0, color='gray', linestyle='-', alpha=0.3)
axes[1].axvline(x=K, color='red', linestyle=':', alpha=0.5, label=f'Strike K={K}')
axes[1].set_xlabel('Spot Price')
axes[1].set_ylabel('Value ($)')
axes[1].set_title('Put Option')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Key Takeaways

1. **Risk-neutral pricing**: Use r - q as drift, not historical returns
2. **Black-Scholes**: Standard framework for European option pricing
3. **Put-call parity**: Fundamental check for any implementation
4. **No arbitrage**: Call ≤ S, Put ≤ K*exp(-rT)

## Next Steps

Continue to **Tier 1: Core Concepts**:
- `01_risk_neutral_valuation.ipynb` - Deep dive into Q-measure
- `02_put_call_parity.ipynb` - Arbitrage relationships
- `03_arbitrage_bounds.ipynb` - Price constraints

## Try It Yourself

**Exercise 1**: Change the volatility to 30%. How does it affect call and put prices?

**Exercise 2**: What happens when S = K (ATM) vs S > K (ITM) vs S < K (OTM)?

**Exercise 3**: Verify that put-call parity still holds with different parameters.